In [40]:
# imports

import requests
import os
import json
from pprint import pprint
from IPython.display import JSON
import pandas as pd
import sqlite3
from sqlite3 import Error

In [41]:
# Will find points of interest in Ladysmith BC, using the Wild Poppy Market latitude, longitude as the initial point of reference

wild_poppy = '48.99400256898083,-123.81788919678714'

# Foursquare

Send a request to Foursquare with a small radius (1000m) with the location of your choice

In [334]:
# Refer to https://developer.foursquare.com/reference/place-search to get the authorization format, url, query parameters
# Send the request through Postman first to try it out, and saved the response to a file.

In [333]:
# Send the request via Python as well.

# Set the keys
foursquare_api = os.environ["FOURSQUARE_API_KEY"]

# Get the url (copied from https://developer.foursquare.com/reference/place-search after entering the Wild Poppy latitude and longitude and the search radius of 1000m)
url = "https://api.foursquare.com/v3/places/search?ll=48.99400256898083%2C-123.81788919678714&radius=1000"

# Get the headers
headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api
}

In [44]:
# Send the request
response_FS = requests.get(url, headers=headers)

In [45]:
# Check the status of the response and print the response text
print(response_FS.status_code)
print(response_FS.text)

200
{"results":[{"fsq_id":"4b9f8746f964a520112937e3","categories":[{"id":13002,"name":"Bakery","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/bakery_","suffix":".png"}},{"id":13065,"name":"Restaurant","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/default_","suffix":".png"}}],"chains":[],"distance":77,"geocodes":{"main":{"latitude":48.993297,"longitude":-123.817749},"roof":{"latitude":48.993297,"longitude":-123.817749}},"link":"/v3/places/4b9f8746f964a520112937e3","location":{"address":"510 1st Ave","country":"CA","cross_street":"","formatted_address":"510 1st Ave, Ladysmith BC V9G 1R3","locality":"Ladysmith","neighborhood":["Cedar"],"postcode":"V9G 1R3","region":"BC"},"name":"Old Town Bakery","related_places":{},"timezone":"America/Vancouver"},{"fsq_id":"4d5dd55d5b276dcb151ef2c5","categories":[{"id":13034,"name":"Café","icon":{"prefix":"https://ss3.4sqi.net/img/categories_v2/food/cafe_","suffix":".png"}},{"id":13049,"name":"Diner","icon":{"prefix":"https:

Parse through the response to get the POI details you want (rating, name, location, etc)

In [46]:
FS_response = response_FS.json()

In [47]:
# Pretty print to view the response more easily

pprint(FS_response)

{'context': {'geo_bounds': {'circle': {'center': {'latitude': 48.99400256898083,
                                                  'longitude': -123.81788919678714},
                                       'radius': 1000}}},
 'results': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
                                       'suffix': '.png'},
                              'id': 13002,
                              'name': 'Bakery'},
                             {'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
                                       'suffix': '.png'},
                              'id': 13065,
                              'name': 'Restaurant'}],
              'chains': [],
              'distance': 77,
              'fsq_id': '4b9f8746f964a520112937e3',
              'geocodes': {'main': {'latitude': 48.993297,
                                    'longitude': -123.817749},
                           'ro

In [77]:
# Use JSON to get a different view of the response

JSON(FS_response)

<IPython.core.display.JSON object>

In [49]:
# Access the name of the first POI

FS_response['results'][0]['name']

'Old Town Bakery'

In [50]:
# identify all the points of interest returned by the Foursquare API within a 1000 m radius of the Wild Poppy

FS_POI = []

for item in FS_response['results']:
    FS_POI.append(item['name'])
    
print(FS_POI)

['Old Town Bakery', 'In the Bean Time', 'RBC Royal Bank', "Mr Popper's Sweet Shoppe", 'High Street Hair', 'Wild Poppy Bistro', 'Appetit', 'The Worldly Gourmet Kitchen Store', 'Salamander Books', 'Fox & Hound']


Put your parsed results into a DataFrame

In [51]:
# try to put the whole parsed response into a dataframe

pd.json_normalize(FS_response)

,results,context.geo_bounds.circle.center.latitude,context.geo_bounds.circle.center.longitude,context.geo_bounds.circle.radius
0,"[{'fsq_id': '4b9f8746f964a520112937e3', 'categ...",48.994003,-123.817889,1000


In [356]:
# not what I want, so access results and put into dataframe

Foursquare_df = pd.json_normalize(FS_response, record_path='results')
Foursquare_df.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,location.address_extended
0,4b9f8746f964a520112937e3,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",[],77,/v3/places/4b9f8746f964a520112937e3,Old Town Bakery,America/Vancouver,48.993297,-123.817749,48.993297,...,510 1st Ave,CA,,"510 1st Ave, Ladysmith BC V9G 1R3",Ladysmith,[Cedar],V9G 1R3,BC,NaN,NaN
1,4d5dd55d5b276dcb151ef2c5,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],42,/v3/places/4d5dd55d5b276dcb151ef2c5,In the Bean Time,America/Vancouver,48.994468,-123.817778,48.994468,...,18 High St,CA,,"18 High St, Ladysmith BC V9G 1B9",Ladysmith,[Cedar],V9G 1B9,BC,NaN,NaN
2,4caf56ad1463a143dbbc93a9,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",[{'id': 'd51a09f0-d890-0132-61d3-7a163eb2a6fc'...,13,/v3/places/4caf56ad1463a143dbbc93a9,RBC Royal Bank,America/Vancouver,48.993577,-123.817534,48.993577,...,527 1 St Ave,CA,,"527 1 St Ave, Ladysmith BC V9G 1A7",Ladysmith,NaN,V9G 1A7,BC,PO Box 1060,NaN
3,543065e3498ece744a874fb3,"[{'id': 17060, 'name': 'Candy Store', 'icon': ...",[],23,/v3/places/543065e3498ece744a874fb3,Mr Popper's Sweet Shoppe,America/Vancouver,48.994100,-123.818307,48.994100,...,32 High St,CA,1st Avenue,"32 High St (1st Avenue), Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,# 1
4,4e3dac44fa76455375bbb10f,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",[],27,/v3/places/4e3dac44fa76455375bbb10f,High Street Hair,America/Vancouver,48.994092,-123.817823,48.994092,...,23 High St,CA,,"23 High St, Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,NaN


In [357]:
Foursquare_df.shape

(10, 21)

In [358]:
Foursquare_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fsq_id                      10 non-null     object 
 1   categories                  10 non-null     object 
 2   chains                      10 non-null     object 
 3   distance                    10 non-null     int64  
 4   link                        10 non-null     object 
 5   name                        10 non-null     object 
 6   timezone                    10 non-null     object 
 7   geocodes.main.latitude      10 non-null     float64
 8   geocodes.main.longitude     10 non-null     float64
 9   geocodes.roof.latitude      10 non-null     float64
 10  geocodes.roof.longitude     10 non-null     float64
 11  location.address            10 non-null     object 
 12  location.country            10 non-null     object 
 13  location.cross_street       9 non-null

In [359]:
Foursquare_df.describe()

,distance,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,geocodes.roof.longitude
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,44.200000,48.993843,-123.817848,48.993843,-123.817848
std,19.228451,0.000386,0.000261,0.000386,0.000261
min,13.000000,48.993297,-123.818307,48.993297,-123.818307
25%,30.750000,48.993608,-123.817865,48.993608,-123.817865
50%,46.000000,48.993762,-123.817807,48.993762,-123.817807
75%,53.000000,48.994098,-123.817756,48.994098,-123.817756
max,77.000000,48.994468,-123.817520,48.994468,-123.817520


In [360]:
# create a separate dataframe for latitudes and longitudes to add to SQL database (didn't end up adding)

latlong_df = Foursquare_df[['name','geocodes.main.latitude','geocodes.main.longitude']]
latlong_df

,name,geocodes.main.latitude,geocodes.main.longitude
0,Old Town Bakery,48.993297,-123.817749
1,In the Bean Time,48.994468,-123.817778
2,RBC Royal Bank,48.993577,-123.817534
3,Mr Popper's Sweet Shoppe,48.994100,-123.818307
4,High Street Hair,48.994092,-123.817823
5,Wild Poppy Bistro,48.993772,-123.817874
6,Appetit,48.993702,-123.818270
7,The Worldly Gourmet Kitchen Store,48.993374,-123.817837
8,Salamander Books,48.993751,-123.817791
9,Fox & Hound,48.994294,-123.817520


# Yelp

Send a request with the same location paramaters (location, radius, etc)

In [68]:
# Send the request through Postman first to try it out, and saved the response to a file.

In [5]:
# Send the request via Python as well.
# Refer to https://www.yelp.com/developers/documentation/v3/authentication to get the API Key, authorization format
# Refer to https://www.yelp.com/developers/documentation/v3/business_search to get the url, query parameters

# Set the keys (had to close and re-open Jupyterlab after adding the Yelp API to Environment Variables to fix KeyError)
yelp_api = 'Bearer '+os.environ["YELP_API_KEY"]

# Get the url (copied from url generated by Postman after entering the Wild Poppy latitude and longitude and the search radius of 1000m)
url = "https://api.yelp.com/v3/businesses/search?latitude=48.99400256898083&longitude=-123.81788919678714&radius=1000"

# Get the headers
headers = {
    "Accept": "application/json",
    "Authorization": yelp_api
}

In [6]:
# Send the request
response = requests.get(url, headers=headers)

In [7]:
# Check the status of the response and print the response text
print(response.status_code)
print(response.text)

200
{"businesses": [{"id": "RxVPUMP2slrLGoLO1jNzCw", "alias": "wild-poppy-market-ladysmith-2", "name": "Wild Poppy Market", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/2ha-5lVXekJyl3-lBGbZfQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/wild-poppy-market-ladysmith-2?adjust_creative=jN6KrDw7JW0Ffv2E0PiPDw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jN6KrDw7JW0Ffv2E0PiPDw", "review_count": 31, "categories": [{"alias": "bakeries", "title": "Bakeries"}, {"alias": "breakfast_brunch", "title": "Breakfast & Brunch"}, {"alias": "sandwiches", "title": "Sandwiches"}], "rating": 4.5, "coordinates": {"latitude": 48.9939504271219, "longitude": -123.818158428481}, "transactions": [], "price": "$$", "location": {"address1": "541 1st Avenue", "address2": "", "address3": "", "city": "Ladysmith", "zip_code": "V9G", "country": "CA", "state": "BC", "display_address": ["541 1st Avenue", "Ladysmith, BC V9G", "Canada"]}, "phone": "+12509248696", "display_pho

Parse through your result and get POI details

In [127]:
Y_response = response.json()

In [128]:
# Pretty print to view the response more easily

pprint(Y_response)

{'businesses': [{'alias': 'wild-poppy-market-ladysmith-2',
                 'categories': [{'alias': 'bakeries', 'title': 'Bakeries'},
                                {'alias': 'breakfast_brunch',
                                 'title': 'Breakfast & Brunch'},
                                {'alias': 'sandwiches', 'title': 'Sandwiches'}],
                 'coordinates': {'latitude': 48.9939504271219,
                                 'longitude': -123.818158428481},
                 'display_phone': '+1 250-924-8696',
                 'distance': 20.48072512998537,
                 'id': 'RxVPUMP2slrLGoLO1jNzCw',
                 'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/2ha-5lVXekJyl3-lBGbZfQ/o.jpg',
                 'is_closed': False,
                 'location': {'address1': '541 1st Avenue',
                              'address2': '',
                              'address3': '',
                              'city': 'Ladysmith',
                              'count

In [129]:
# Use JSON to get a different view of the response

JSON(Y_response)

<IPython.core.display.JSON object>

In [130]:
# Access the name of the first POI

Y_response['businesses'][0]['name']

'Wild Poppy Market'

In [131]:
# identify all the points of interest returned by the Yelp API within a 1000 m radius of the Wild Poppy

Y_POI = []

for item in Y_response['businesses']:
    Y_POI.append(item['name'])
    
print(Y_POI)

['Wild Poppy Market', 'Old Town Bakery', 'Spice Hut', 'Fox And Hounds', 'Plantitude', 'Maya Norte', 'Appetit Food for Thought', 'In the Beantime Cafe', 'Kaede Sushi & Grill', 'Oyster Bay Cafe', 'Roberts Street Pizza', 'Phoenix Restaurant', 'Ironworks Cafe and Crêperie', "Zack's", 'Mr. Teriyaki and Miss Sushi', 'Tim Hortons', 'The Wigwam Restaurant', 'Dragon City Restaurant', 'Timberland Pub', 'Taco Revolution']


Put your parsed results into a DataFrame

In [16]:
# try to put the whole parsed response into a dataframe

pd.json_normalize(Y_response)

,businesses,total,region.center.longitude,region.center.latitude
0,"[{'id': 'RxVPUMP2slrLGoLO1jNzCw', 'alias': 'wi...",20,-123.817889,48.994003


In [374]:
# not what I want, so access businesses and put into dataframe

Org_Yelp_df = pd.json_normalize(Y_response, record_path='businesses')
Org_Yelp_df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,RxVPUMP2slrLGoLO1jNzCw,wild-poppy-market-ladysmith-2,Wild Poppy Market,https://s3-media4.fl.yelpcdn.com/bphoto/2ha-5l...,False,https://www.yelp.com/biz/wild-poppy-market-lad...,31,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,[],...,48.993950,-123.818158,541 1st Avenue,,,Ladysmith,V9G,CA,BC,"[541 1st Avenue, Ladysmith, BC V9G, Canada]"
1,NolpyYCEneVkMxOfO66HEQ,old-town-bakery-ladysmith,Old Town Bakery,https://s3-media2.fl.yelpcdn.com/bphoto/juE5xk...,False,https://www.yelp.com/biz/old-town-bakery-ladys...,33,"[{'alias': 'bakeries', 'title': 'Bakeries'}]",4.5,[],...,48.993309,-123.817640,510 1st Avenue,,,Ladysmith,V9G 1R3,CA,BC,"[510 1st Avenue, Ladysmith, BC V9G 1R3, Canada]"
2,eKNuWQkubnIGoUZAIFFg8Q,spice-hut-ladysmith-2,Spice Hut,https://s3-media2.fl.yelpcdn.com/bphoto/s2VSqp...,False,https://www.yelp.com/biz/spice-hut-ladysmith-2...,19,"[{'alias': 'indpak', 'title': 'Indian'}]",4.5,[],...,48.994116,-123.816207,510 Esplanade Avenue,,,Ladysmith,V9G,CA,BC,"[510 Esplanade Avenue, Ladysmith, BC V9G, Canada]"
3,5g8G8DnCI-tBnLiUxUEXzA,fox-and-hounds-ladysmith,Fox And Hounds,https://s3-media1.fl.yelpcdn.com/bphoto/XPnymM...,False,https://www.yelp.com/biz/fox-and-hounds-ladysm...,19,"[{'alias': 'pubs', 'title': 'Pubs'}]",4.0,[],...,48.994318,-123.817674,11 High Street,,,Ladysmith,V9G,CA,BC,"[11 High Street, Ladysmith, BC V9G, Canada]"
4,0inqHeKSgGb0ju40sPviSQ,plantitude-ladysmith,Plantitude,https://s3-media2.fl.yelpcdn.com/bphoto/93UFHT...,False,https://www.yelp.com/biz/plantitude-ladysmith?...,7,"[{'alias': 'vegan', 'title': 'Vegan'}]",5.0,[],...,48.992936,-123.816630,431 1st Avenue,,None,Ladysmith,V9G 1A9,CA,BC,"[431 1st Avenue, Ladysmith, BC V9G 1A9, Canada]"


In [375]:
Org_Yelp_df.shape

(20, 24)

In [376]:
Org_Yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        20 non-null     object 
 1   alias                     20 non-null     object 
 2   name                      20 non-null     object 
 3   image_url                 20 non-null     object 
 4   is_closed                 20 non-null     bool   
 5   url                       20 non-null     object 
 6   review_count              20 non-null     int64  
 7   categories                20 non-null     object 
 8   rating                    20 non-null     float64
 9   transactions              20 non-null     object 
 10  price                     11 non-null     object 
 11  phone                     20 non-null     object 
 12  display_phone             20 non-null     object 
 13  distance                  20 non-null     float64
 14  coordinates.

In [377]:
Org_Yelp_df.describe()

,review_count,rating,distance,coordinates.latitude,coordinates.longitude
count,20.000000,20.00000,20.000000,20.000000,20.000000
mean,10.350000,3.97500,2061.531134,48.978514,-123.807110
std,9.022049,0.89553,8330.870661,0.068645,0.047886
min,1.000000,1.50000,20.480725,48.686989,-123.823662
25%,4.750000,3.50000,73.153584,48.993120,-123.818336
50%,7.000000,4.25000,144.596891,48.993625,-123.817657
75%,13.250000,4.50000,273.674498,48.994336,-123.815778
max,33.000000,5.00000,37446.227935,48.997681,-123.603912


In [155]:
business_list = []

for business in Y_response['businesses']:
    # print(business['name'])
    # print(', '.join(cat['title'] for cat in business['categories']))
    # print(business['display_phone'])
    # print(', '.join(business['location']['display_address']))
    # print(business['price'])
    # print(business['rating'])
    # print(business['review_count'])
    business_list.append({
        'name':business['name'],
        'categories':', '.join(cat['title'] for cat in business['categories']),
        'display_phone':business['display_phone'],
        'address':', '.join(business['location']['display_address']),
        'price':business['price'] if business.get('price') else 'No Price',
        'rating':business['rating'],
        'review_count':business['review_count']
    })
Yelp_df = pd.DataFrame(business_list)
Yelp_df

,name,categories,display_phone,address,price,rating,review_count
0,Wild Poppy Market,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31
1,Old Town Bakery,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33
2,Spice Hut,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19
3,Fox And Hounds,Pubs,+1 250-924-1747,"11 High Street, Ladysmith, BC V9G, Canada",$$,4.0,19
4,Plantitude,Vegan,+1 250-924-1201,"431 1st Avenue, Ladysmith, BC V9G 1A9, Canada",No Price,5.0,7
5,Maya Norte,"Spanish, Mexican",+1 250-924-1300,"18 Roberts Street, Ladysmith, BC V9G, Canada",No Price,4.5,12
6,Appetit Food for Thought,"Thai, Asian Fusion, Breakfast & Brunch",+1 250-924-1321,"534 1st Avenue, Ladysmith, BC V9G 1A9, Canada",$$,4.5,8
7,In the Beantime Cafe,Coffee & Tea,+1 250-245-2305,"18 High St, Ladysmith, BC V9G 1B9, Canada",$$,4.0,13
8,Kaede Sushi & Grill,"Japanese, Sushi Bars",+1 250-924-1166,"734 Esplanade Avenue, Ladysmith, BC V9G 1A1, C...",No Price,4.5,7
9,Oyster Bay Cafe,Cafes,+1 250-924-2245,"611 Oyster Bay Drive, Ladysmith Maritime Socie...",$$,4.5,7


In [336]:
# add a primary key for POI name

Yelp_df['poi_id'] = range(1,(len(Yelp_df['name'])+1))
Yelp_df.head()

,name,categories,display_phone,address,price,rating,review_count,poi_id
0,Wild Poppy Market,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
1,Old Town Bakery,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2
2,Spice Hut,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3
3,Fox And Hounds,Pubs,+1 250-924-1747,"11 High Street, Ladysmith, BC V9G, Canada",$$,4.0,19,4
4,Plantitude,Vegan,+1 250-924-1201,"431 1st Avenue, Ladysmith, BC V9G 1A9, Canada",No Price,5.0,7,5


In [191]:
business_df = Yelp_df[['name','price','rating','review_count']]

In [192]:
business_df.head()

,name,price,rating,review_count
0,Wild Poppy Market,$$,4.5,31
1,Old Town Bakery,$$,4.5,33
2,Spice Hut,$$,4.5,19
3,Fox And Hounds,$$,4.0,19
4,Plantitude,No Price,5.0,7


In [337]:
# create a separate dataframe that expands the category list

categories_df = Yelp_df.set_index('name')\
  .categories.str.split(',', expand=True)\
  .stack()\
  .reset_index('name')\
  .rename(columns={0:'category'})\
  .reset_index(drop=True)
categories_df

,name,category
0,Wild Poppy Market,Bakeries
1,Wild Poppy Market,Breakfast & Brunch
2,Wild Poppy Market,Sandwiches
3,Old Town Bakery,Bakeries
4,Spice Hut,Indian
5,Fox And Hounds,Pubs
6,Plantitude,Vegan
7,Maya Norte,Spanish
8,Maya Norte,Mexican
9,Appetit Food for Thought,Thai


In [338]:
# When I created a category id table for the database, I realized there were 'duplicate' categories that had a leading space before the category name. In order to successfully merge this category_df table with the category table I created in my database, I needed to strip these leading spaces.

categories_df['category'] = categories_df['category'].str.strip()

In [159]:
location_df = Yelp_df[['name','address']]
location_df.head()

,name,address
0,Wild Poppy Market,"541 1st Avenue, Ladysmith, BC V9G, Canada"
1,Old Town Bakery,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada"
2,Spice Hut,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada"
3,Fox And Hounds,"11 High Street, Ladysmith, BC V9G, Canada"
4,Plantitude,"431 1st Avenue, Ladysmith, BC V9G 1A9, Canada"


In [193]:
phone_df = Yelp_df[['display_phone','poi_id']]
phone_df.head()

,display_phone,poi_id
0,+1 250-924-8696,1
1,+1 250-245-2531,2
2,+1 250-924-8222,3
3,+1 250-924-1747,4
4,+1 250-924-1201,5


# Google (stretch)

Use the same process as the first two APIs

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [26]:
# define a function create_connection() that accepts the path to the SQLite database
def create_connection(path):
    
    connection = None
    
    try:
        # use .connect() from the sqlite3 module, with the database path as a parameter. If the database exists at the specified location, then a connection to the database is established. Otherwise, a new database is created at the specified location, and a connection is established
        connection = sqlite3.connect(path)
        
        # print the status of the successful database connection.
        print("Connection to SQLite DB successful")
    
    # catches any exception that might be thrown if .connect() fails to establish a connection
    except Error as e:
        # displays the error message in the console.
        print(f"The error '{e}' occurred")

    return connection

In [105]:
# passed the desired file path of the database to the create_connection function to create the database and connect to it

connection = create_connection("../data/mini-project-2.sqlite")

Connection to SQLite DB successful


In [141]:
# function to execute the given query and prints an error message if necessary
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

### Create the POI table

In [ ]:
#define the poi table

In [165]:
create_poi_table = """
CREATE TABLE IF NOT EXISTS poi (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  name TEXT NOT NULL,
  price TEXT,
  rating REAL,
  review_count INTEGER
);
"""

In [ ]:
# create the poi table

In [166]:
execute_query(connection, create_poi_table)

Query executed successfully


In [237]:
# get the values for the poi table

value_list = []

for i in range(len(business_df)):
    value_list.append(tuple(business_df.loc[i]))
    
value_list

[('Wild Poppy Market', '$$', 4.5, 31),
 ('Old Town Bakery', '$$', 4.5, 33),
 ('Spice Hut', '$$', 4.5, 19),
 ('Fox And Hounds', '$$', 4.0, 19),
 ('Plantitude', 'No Price', 5.0, 7),
 ('Maya Norte', 'No Price', 4.5, 12),
 ('Appetit Food for Thought', '$$', 4.5, 8),
 ('In the Beantime Cafe', '$$', 4.0, 13),
 ('Kaede Sushi & Grill', 'No Price', 4.5, 7),
 ('Oyster Bay Cafe', '$$', 4.5, 7),
 ('Roberts Street Pizza', '$$', 3.5, 14),
 ('Phoenix Restaurant', '$$', 4.5, 5),
 ('Ironworks Cafe and Crêperie', 'No Price', 4.0, 8),
 ("Zack's", 'No Price', 3.5, 4),
 ('Mr. Teriyaki and Miss Sushi', '$', 4.0, 7),
 ('Tim Hortons', 'No Price', 3.5, 3),
 ('The Wigwam Restaurant', 'No Price', 1.5, 2),
 ('Dragon City Restaurant', 'No Price', 2.0, 5),
 ('Timberland Pub', '$', 3.5, 2),
 ('Taco Revolution', 'No Price', 5.0, 1)]

In [168]:
create_poi = """
INSERT INTO
  poi (name, price, rating, review_count)
VALUES
  ('Wild Poppy Market', '$$', 4.5, 31),
 ('Old Town Bakery', '$$', 4.5, 33),
 ('Spice Hut', '$$', 4.5, 19),
 ('Fox And Hounds', '$$', 4.0, 19),
 ('Plantitude', 'No Price', 5.0, 7),
 ('Maya Norte', 'No Price', 4.5, 12),
 ('Appetit Food for Thought', '$$', 4.5, 8),
 ('In the Beantime Cafe', '$$', 4.0, 13),
 ('Kaede Sushi & Grill', 'No Price', 4.5, 7),
 ('Oyster Bay Cafe', '$$', 4.5, 7),
 ('Roberts Street Pizza', '$$', 3.5, 14),
 ('Phoenix Restaurant', '$$', 4.5, 5),
 ('Ironworks Cafe and Crêperie', 'No Price', 4.0, 8),
 ("Zack's", 'No Price', 3.5, 4),
 ('Mr. Teriyaki and Miss Sushi', '$', 4.0, 7),
 ('Tim Hortons', 'No Price', 3.5, 3),
 ('The Wigwam Restaurant', 'No Price', 1.5, 2),
 ('Dragon City Restaurant', 'No Price', 2.0, 5),
 ('Timberland Pub', '$', 3.5, 2),
 ('Taco Revolution', 'No Price', 5.0, 1);
"""

execute_query(connection, create_poi)

Query executed successfully


### Create a phone_number table & reference the POI table

In [176]:
# create phone_number table

create_phone_number_table = """
CREATE TABLE IF NOT EXISTS phone_number (
  display_phone TEXT NOT NULL,
  poi_id INTEGER NOT NULL,
  FOREIGN KEY (poi_id) REFERENCES poi (id)
);
"""

In [177]:
execute_query(connection, create_phone_number_table)

Query executed successfully


In [194]:
value_list_phone = []

for i in range(len(phone_df)):
    value_list_phone.append(tuple(phone_df.loc[i]))
    
value_list_phone

[('+1 250-924-8696', 1),
 ('+1 250-245-2531', 2),
 ('+1 250-924-8222', 3),
 ('+1 250-924-1747', 4),
 ('+1 250-924-1201', 5),
 ('+1 250-924-1300', 6),
 ('+1 250-924-1321', 7),
 ('+1 250-245-2305', 8),
 ('+1 250-924-1166', 9),
 ('+1 250-924-2245', 10),
 ('+1 250-245-1119', 11),
 ('+1 250-245-3263', 12),
 ('+1 250-912-0064', 13),
 ('+1 250-245-2749', 14),
 ('+1 250-924-2221', 15),
 ('+1 250-245-2303', 16),
 ('+1 250-245-3433', 17),
 ('+1 250-245-8689', 18),
 ('+1 250-245-0800', 19),
 ('+1 250-244-6492', 20)]

In [196]:
create_phone_number = """
INSERT INTO
  phone_number (display_phone, poi_id)
VALUES
  ('+1 250-924-8696', 1),
 ('+1 250-245-2531', 2),
 ('+1 250-924-8222', 3),
 ('+1 250-924-1747', 4),
 ('+1 250-924-1201', 5),
 ('+1 250-924-1300', 6),
 ('+1 250-924-1321', 7),
 ('+1 250-245-2305', 8),
 ('+1 250-924-1166', 9),
 ('+1 250-924-2245', 10),
 ('+1 250-245-1119', 11),
 ('+1 250-245-3263', 12),
 ('+1 250-912-0064', 13),
 ('+1 250-245-2749', 14),
 ('+1 250-924-2221', 15),
 ('+1 250-245-2303', 16),
 ('+1 250-245-3433', 17),
 ('+1 250-245-8689', 18),
 ('+1 250-245-0800', 19),
 ('+1 250-244-6492', 20);
"""

execute_query(connection, create_phone_number)

Query executed successfully


### Create the category table

In [198]:
# define the category table

In [283]:
create_category_table = """
CREATE TABLE IF NOT EXISTS category (
  id INTEGER PRIMARY KEY AUTOINCREMENT,
  category TEXT NOT NULL
);
"""

In [284]:
# create the category table

In [285]:
execute_query(connection, create_category_table)

Query executed successfully


In [344]:
# strip leading spaces to avoid duplicates in the category table

category_only = categories_df['category'].unique()

category_only_new = []

for item in category_only:
    category_only_new.append(item.lstrip())

category_only_new = set(category_only_new)
category_only_new

{'Asian Fusion',
 'Bakeries',
 'Breakfast & Brunch',
 'Burgers',
 'Cafes',
 'Chinese',
 'Coffee & Tea',
 'Creperies',
 'Food Trucks',
 'Indian',
 'Italian',
 'Japanese',
 'Lounges',
 'Mexican',
 'Pizza',
 'Pubs',
 'Sandwiches',
 'Spanish',
 'Sushi Bars',
 'Thai',
 'Vegan',
 'Vegetarian',
 'Vietnamese'}

In [287]:
create_category = """
INSERT INTO
  category (category)
VALUES
  ('Asian Fusion'),
 ('Bakeries'),
 ('Breakfast & Brunch'),
 ('Burgers'),
 ('Cafes'),
 ('Chinese'),
 ('Coffee & Tea'),
 ('Creperies'),
 ('Food Trucks'),
 ('Indian'),
 ('Italian'),
 ('Japanese'),
 ('Lounges'),
 ('Mexican'),
 ('Pizza'),
 ('Pubs'),
 ('Sandwiches'),
 ('Spanish'),
 ('Sushi Bars'),
 ('Thai'),
 ('Vegan'),
 ('Vegetarian'),
 ('Vietnamese');
"""

execute_query(connection, create_category)

Query executed successfully


In [346]:
# delete_table = "DROP TABLE category"
# execute_query(connection, delete_table)

In [345]:
# delete_table = "DROP TABLE Businesses"
# execute_query(connection, delete_table)

### Create the POI_category table to link the category and POI tables

In [ ]:
# define the POI_category table

In [ ]:
# (didn't end up creating the table this way)

# create_POI_category_table = """
# CREATE TABLE IF NOT EXISTS POI_category (
#   poi_id INTEGER NOT NULL,
#   category_id INTEGER NOT NULL,
#   FOREIGN KEY (poi_id) REFERENCES poi (id) FOREIGN KEY (category_id) REFERENCES category (id)
# );
# """

In [349]:
# want to create a table that contains the category_id and the poi_id in one table.

In [350]:
# step 1 - get the category_id generated by sql

In [288]:
category_from_sql = pd.read_sql('select * from category', connection)

In [353]:
category_from_sql.head()

,id,category
0,1,Asian Fusion
1,2,Bakeries
2,3,Breakfast & Brunch
3,4,Burgers
4,5,Cafes


In [351]:
# step 2 - merge the original categories_df with the Yelp_df to match the categories with poi_ids

In [309]:
poi_category_df = pd.merge(categories_df, Yelp_df, on='name', how='outer')
poi_category_df.head()

,name,category,categories,display_phone,address,price,rating,review_count,poi_id
0,Wild Poppy Market,Bakeries,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
1,Wild Poppy Market,Breakfast & Brunch,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
2,Wild Poppy Market,Sandwiches,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
3,Old Town Bakery,Bakeries,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2
4,Spice Hut,Indian,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3


In [ ]:
# step 3 - merge the table created in step 2 with the table from step 1 to get the corresponding category ids.

In [348]:
poi_category_new_df = pd.merge(poi_category_df, category_from_sql, on='category', how='left')
poi_category_new_df.head()

,name,category,categories,display_phone,address,price,rating,review_count,poi_id,id
0,Wild Poppy Market,Bakeries,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,2
1,Wild Poppy Market,Breakfast & Brunch,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,3
2,Wild Poppy Market,Sandwiches,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,17
3,Old Town Bakery,Bakeries,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2,2
4,Spice Hut,Indian,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3,10


In [352]:
# step 4 - create a new dataframe that contains only the category id and poi id.

In [354]:
values = poi_category_new_df[['id','poi_id']]
values.head()

,id,poi_id
0,2,1
1,3,1
2,17,1
3,2,2
4,10,3


In [ ]:
# create the POI_category intersection table

In [315]:
values.to_sql('POI_category', connection, if_exists="replace", index=False)

34

# Analysis

### Get the top 10 restaurants according to their rating

In [321]:
Yelp_df.head()

,name,categories,display_phone,address,price,rating,review_count,poi_id
0,Wild Poppy Market,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
1,Old Town Bakery,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2
2,Spice Hut,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3
3,Fox And Hounds,Pubs,+1 250-924-1747,"11 High Street, Ladysmith, BC V9G, Canada",$$,4.0,19,4
4,Plantitude,Vegan,+1 250-924-1201,"431 1st Avenue, Ladysmith, BC V9G 1A9, Canada",No Price,5.0,7,5


In [330]:
top10 = Yelp_df[['name','rating']].sort_values(by='rating', ascending = False).nlargest(10, 'rating', keep='all')

In [331]:
top10

,name,rating
19,Taco Revolution,5.0
4,Plantitude,5.0
8,Kaede Sushi & Grill,4.5
11,Phoenix Restaurant,4.5
1,Old Town Bakery,4.5
9,Oyster Bay Cafe,4.5
0,Wild Poppy Market,4.5
6,Appetit Food for Thought,4.5
5,Maya Norte,4.5
2,Spice Hut,4.5


### Compare number of results from FourSquare API versus Yelp API

In [364]:
Yelp_df.head()

,name,categories,display_phone,address,price,rating,review_count,poi_id
0,Wild Poppy Market,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1
1,Old Town Bakery,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2
2,Spice Hut,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3
3,Fox And Hounds,Pubs,+1 250-924-1747,"11 High Street, Ladysmith, BC V9G, Canada",$$,4.0,19,4
4,Plantitude,Vegan,+1 250-924-1201,"431 1st Avenue, Ladysmith, BC V9G 1A9, Canada",No Price,5.0,7,5


In [365]:
Foursquare_df.head()

,fsq_id,categories,chains,distance,link,name,timezone,geocodes.main.latitude,geocodes.main.longitude,geocodes.roof.latitude,...,location.address,location.country,location.cross_street,location.formatted_address,location.locality,location.neighborhood,location.postcode,location.region,location.po_box,location.address_extended
0,4b9f8746f964a520112937e3,"[{'id': 13002, 'name': 'Bakery', 'icon': {'pre...",[],77,/v3/places/4b9f8746f964a520112937e3,Old Town Bakery,America/Vancouver,48.993297,-123.817749,48.993297,...,510 1st Ave,CA,,"510 1st Ave, Ladysmith BC V9G 1R3",Ladysmith,[Cedar],V9G 1R3,BC,NaN,NaN
1,4d5dd55d5b276dcb151ef2c5,"[{'id': 13034, 'name': 'Café', 'icon': {'prefi...",[],42,/v3/places/4d5dd55d5b276dcb151ef2c5,In the Bean Time,America/Vancouver,48.994468,-123.817778,48.994468,...,18 High St,CA,,"18 High St, Ladysmith BC V9G 1B9",Ladysmith,[Cedar],V9G 1B9,BC,NaN,NaN
2,4caf56ad1463a143dbbc93a9,"[{'id': 11045, 'name': 'Bank', 'icon': {'prefi...",[{'id': 'd51a09f0-d890-0132-61d3-7a163eb2a6fc'...,13,/v3/places/4caf56ad1463a143dbbc93a9,RBC Royal Bank,America/Vancouver,48.993577,-123.817534,48.993577,...,527 1 St Ave,CA,,"527 1 St Ave, Ladysmith BC V9G 1A7",Ladysmith,NaN,V9G 1A7,BC,PO Box 1060,NaN
3,543065e3498ece744a874fb3,"[{'id': 17060, 'name': 'Candy Store', 'icon': ...",[],23,/v3/places/543065e3498ece744a874fb3,Mr Popper's Sweet Shoppe,America/Vancouver,48.994100,-123.818307,48.994100,...,32 High St,CA,1st Avenue,"32 High St (1st Avenue), Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,# 1
4,4e3dac44fa76455375bbb10f,"[{'id': 11064, 'name': 'Hair Salon', 'icon': {...",[],27,/v3/places/4e3dac44fa76455375bbb10f,High Street Hair,America/Vancouver,48.994092,-123.817823,48.994092,...,23 High St,CA,,"23 High St, Ladysmith BC V9G 1B9",Ladysmith,NaN,V9G 1B9,BC,NaN,NaN


In [363]:
len(Foursquare_df['name'])

10

In [372]:
len(business_df['name'])

20

The FourSquare API only returned 10 points of interest, whereas the Yelp API returned 20 points of interest in the same search radius

### Compare breadth of results from FourSquare API versus Yelp API

In [383]:
Foursquare_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fsq_id                      10 non-null     object 
 1   categories                  10 non-null     object 
 2   chains                      10 non-null     object 
 3   distance                    10 non-null     int64  
 4   link                        10 non-null     object 
 5   name                        10 non-null     object 
 6   timezone                    10 non-null     object 
 7   geocodes.main.latitude      10 non-null     float64
 8   geocodes.main.longitude     10 non-null     float64
 9   geocodes.roof.latitude      10 non-null     float64
 10  geocodes.roof.longitude     10 non-null     float64
 11  location.address            10 non-null     object 
 12  location.country            10 non-null     object 
 13  location.cross_street       9 non-null

In [380]:
len(Foursquare_df.columns)

21

In [384]:
Org_Yelp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        20 non-null     object 
 1   alias                     20 non-null     object 
 2   name                      20 non-null     object 
 3   image_url                 20 non-null     object 
 4   is_closed                 20 non-null     bool   
 5   url                       20 non-null     object 
 6   review_count              20 non-null     int64  
 7   categories                20 non-null     object 
 8   rating                    20 non-null     float64
 9   transactions              20 non-null     object 
 10  price                     11 non-null     object 
 11  phone                     20 non-null     object 
 12  display_phone             20 non-null     object 
 13  distance                  20 non-null     float64
 14  coordinates.

In [381]:
len(Org_Yelp_df.columns)

24

Yelp provides a wider variety of information for each point of interest. It has 24 items describing each point of interest, including many that are not provided by the FourSquare API (rating, review_count, price informatiom, etc.)

### Which category of restaurant has the best ratings in Ladysmith?

In [386]:
# used merged table that shows the individual categories for each restaurant, in addition to each restaurant's rating.

poi_category_new_df.head()

,name,category,categories,display_phone,address,price,rating,review_count,poi_id,id
0,Wild Poppy Market,Bakeries,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,2
1,Wild Poppy Market,Breakfast & Brunch,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,3
2,Wild Poppy Market,Sandwiches,"Bakeries, Breakfast & Brunch, Sandwiches",+1 250-924-8696,"541 1st Avenue, Ladysmith, BC V9G, Canada",$$,4.5,31,1,17
3,Old Town Bakery,Bakeries,Bakeries,+1 250-245-2531,"510 1st Avenue, Ladysmith, BC V9G 1R3, Canada",$$,4.5,33,2,2
4,Spice Hut,Indian,Indian,+1 250-924-8222,"510 Esplanade Avenue, Ladysmith, BC V9G, Canada",$$,4.5,19,3,10


In [404]:
poi_category_new_df.groupby('category').agg(average_rating = ('rating','mean')).nlargest(5, 'average_rating', keep = 'all')

,average_rating
category,
Food Trucks,5.00
Vegan,5.00
Mexican,4.75
Asian Fusion,4.50
Bakeries,4.50
Breakfast & Brunch,4.50
Indian,4.50
Spanish,4.50
Thai,4.50


Food Trucks, Vegan, and Mexican restaurants had the highest ratings

### Most common restaurant type in Ladysmith?

In [407]:
poi_category_new_df['category'].value_counts().head(1)

Chinese    3
Name: category, dtype: int64

In [401]:
poi_category_new_df['category'].value_counts() > 1

Chinese                True
Bakeries               True
Mexican                True
Cafes                  True
Sushi Bars             True
Breakfast & Brunch     True
Coffee & Tea           True
Japanese               True
Pubs                   True
Sandwiches             True
Thai                  False
Asian Fusion          False
Spanish               False
Vegan                 False
Indian                False
Pizza                 False
Italian               False
Creperies             False
Burgers               False
Lounges               False
Vietnamese            False
Vegetarian            False
Food Trucks           False
Name: category, dtype: bool

Chinese restaurants are the most common type of restaurant in Ladysmith.

In [409]:
# look at highest quality, most reviews, cheapest restaurant?

# Travelling Salesman Problem (stretch)

If you have time, follow the steps in the [ortools tutorial](https://developers.google.com/optimization/routing/tsp) using Google's [Directions API](https://developers.google.com/maps/documentation/directions/start).